# Intel D435 prediction cam codes

In [ ]:
# Institution: Carleton University
# Course: OSS4009 Capstone 
# Term: F22 - W23
#
# Filename: 0 - Pre0 - Intel_VideoCam - saving images.ipynb
#
# Students: Adam Thompson, Philippe Beaulieu
# Advisor:  Dr. Marzieh Amini
#
# Description: This program will save the images from the Intel D435 RGB and Depth stream
#              in the proper folder, starting at img0.jpg and increaing in numbers.
#
#              The program will trat to warm up the camera, when the image is stable you hit 'c' 
#              to continue with the gesture record, you will be prompted for the start of each action
#              before before starting the record process by hitting any key.
#
#              The amount of images to record can be adjusted in the setup.
#
#              The foldername will indicate the action or gesture made, keeping it simple
#              you have to create empty folders with identical name for both RGB and DEPTH.
#              The amount of folder you have depend on the amount or gesture you want to
#              train your model for.
#
#              The folder hierarchy is important to save the images, it is as follow:
#
#    -TRAIN
#       -DEPTH
#          -folder0
#          -folder1
#          -...
#       -RGB
#          -folder0
#          -folder1
#          -...
#    -TEST -> follow the same structure as train
#

setup

In [1]:
import pyrealsense2 as rs
import cv2

import matplotlib.pyplot as plt
import numpy as np
import PIL

import pathlib
import os
import json

import warnings
warnings.filterwarnings('ignore')

#img_width  = 480
#img_height = 270

sm_img_width  = 160
sm_img_height  = 120

# set amount of images you want per category
img_amount = 600

prepare folder data

In [5]:
path_rgb   = "TRAIN/RGB"    # enter the RGB image dataset path
path_depth = "TRAIN/DEPTH"  # enter the DEPTH image dataset path

class_name = []
for folder in os.listdir(path_rgb):
    class_name.append(folder)
print("Class Name:",class_name)

#with open("class_name.json", 'r') as f:
#    class_name = json.load(f)

Class Name: ['1finger', '2finger', 'fist', 'palm', 'thumb_up']


Using Intel D435 as cam input for images

In [6]:
# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()

# Get device product line for setting a supporting resolution
pipeline_wrapper = rs.pipeline_wrapper(pipeline)
pipeline_profile = config.resolve(pipeline_wrapper)
device = pipeline_profile.get_device()
device_product_line = str(device.get_info(rs.camera_info.product_line))

found_rgb = False
for s in device.sensors:
    if s.get_info(rs.camera_info.name) == 'RGB Camera':
        found_rgb = True
        break
if not found_rgb:
    print("This code requires Depth camera with Color sensor")
    exit(0)

config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)   # 1/2-> 320,240  1/4-> 160,120

if device_product_line == 'L500':
    config.enable_stream(rs.stream.color, 960, 540, rs.format.bgr8, 30)
else:
    config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

# Start streaming
pipeline.start(config)

try:

    
    # section created to warm the camera up, 'q' to continue to the image acwuisition section
    while True:
        token = True
        while token:
            # Wait for a coherent pair of frames: depth and color
            frames = pipeline.wait_for_frames()
            depth_frame = frames.get_depth_frame()
            color_frame = frames.get_color_frame()
            if not depth_frame or not color_frame:
                continue
            token = False

        decimation = rs.decimation_filter()
        decimation.set_option(rs.option.filter_magnitude, 1)
        decimated_depth = decimation.process(depth_frame)
        
        spatial = rs.spatial_filter()
        spatial.set_option(rs.option.filter_magnitude, 5)
        spatial.set_option(rs.option.filter_smooth_alpha, 1)
        spatial.set_option(rs.option.filter_smooth_delta, 50)
        filtered_depth = spatial.process(decimated_depth)
            
        hole_filling = rs.hole_filling_filter()
        filled_depth = hole_filling.process(depth_frame)
            
        # Convert images to numpy arrays
        depth_image = np.asanyarray(filled_depth.get_data())
        color_image = np.asanyarray(color_frame.get_data())

        # Apply colormap on depth image (image must be converted to 8-bit per pixel first)
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)

        depth_colormap_dim = depth_colormap.shape
        color_colormap_dim = color_image.shape

        # If depth and color resolutions are different, resize color image to match depth image for display
        if depth_colormap_dim != color_colormap_dim:
            resized_color_image = cv2.resize(color_image, dsize=(depth_colormap_dim[1], depth_colormap_dim[0]), interpolation=cv2.INTER_AREA)
            images = np.hstack((resized_color_image, depth_colormap))
        else:
            images = np.hstack((color_image, depth_colormap))
        # Show images 
        cv2.namedWindow('RealSense', cv2.WINDOW_AUTOSIZE)
        cv2.imshow('RealSense', images)
        key = cv2.waitKey(10)
        if key & 0xFF == ord('q') or key == 27:
            break
 

    # images acquisition
    # section that will start capturin the RGB and depth images
    for folder in class_name:
        print("Press a key to start the ", folder)
        cv2.waitKey(0)
        
        for img_nb in range(img_amount):

            token = True
            while token:
                # Wait for a coherent pair of frames: depth and color
                frames = pipeline.wait_for_frames()
                depth_frame = frames.get_depth_frame()
                color_frame = frames.get_color_frame()
                if not depth_frame or not color_frame:
                    continue
                token = False

            hole_filling = rs.hole_filling_filter()
            filled_depth = hole_filling.process(depth_frame)
                
            # Convert images to numpy arrays
            depth_image = np.asanyarray(filled_depth.get_data())
            color_image = np.asanyarray(color_frame.get_data())

            # Apply colormap on depth image (image must be converted to 8-bit per pixel first)
            depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.45), cv2.COLORMAP_JET)

            depth_colormap_dim = depth_colormap.shape
            color_colormap_dim = color_image.shape

            # If depth and color resolutions are different, resize color image to match depth image for display
            if depth_colormap_dim != color_colormap_dim:
                resized_color_image = cv2.resize(color_image, dsize=(depth_colormap_dim[1], depth_colormap_dim[0]), interpolation=cv2.INTER_AREA)
                images = np.hstack((resized_color_image, depth_colormap))
            else:
                images = np.hstack((color_image, depth_colormap))


            cv2.putText(images, str(img_nb), (50,50), cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,0),3)
            # Show images 
            cv2.namedWindow('RealSense', cv2.WINDOW_AUTOSIZE)
            cv2.imshow('RealSense', images)

            # resize images to 160, 120
            small_color_img = cv2.resize(color_image, (sm_img_width, sm_img_height), interpolation=cv2.INTER_AREA)
            small_depth_img = cv2.resize(depth_colormap, (sm_img_width, sm_img_height), interpolation=cv2.INTER_AREA)
            # save images in the right folder
            # filename = path_rgb + "/" + folder + "/"  + "img" + str(img_nb) + ".jpg"
            cv2.imwrite(path_rgb + "/" + folder + "/"  + "img" + str(img_nb) + ".jpg", small_color_img)
            cv2.imwrite(path_depth + "/" + folder + "/" + "img" + str(img_nb) + ".jpg", small_depth_img)

            key = cv2.waitKey(10)
            if key & 0xFF == ord('q') or key == 27:
                cv2.destroyAllWindows()
                break                

finally:

    cv2.destroyAllWindows()
    # Stop streaming
    pipeline.stop()
    

NameError: name 'rs' is not defined